In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# builtins
import locale
import math
import glob
import pathlib
import functools
import logging
import time
import datetime

# numerical stuff
#import pickle5 as pickle
import tables

import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, Lambda
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Input, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
#from tensorflow.python import ipu

#import libpvti as pvti

# plotting
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO)


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.64.129.218:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.64.129.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.129.218:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
data_path_train = 'gs://bathy_sample/processed/20211013/train_data_mask_no_schematic'
data_path_test = 'gs://bathy_sample/processed/20211013/test_data_mask_no_schematic'
#meta_data_path = '/mnt/poddata/data/bathy-emodnet-a-runs.h5'
#all_checkpoints_path = 'gs://bathy_sample/dnn/checkpoints'
all_checkpoints_path = '/content/drive/MyDrive/DeepLearning/Benchmark/checkpoints'
model_name = 'guus-2d-mlp-cnn-v0.4'
model_path = '/content/drive/MyDrive/DeepLearning/Benchmark/mask_models'
checkpoints_path = all_checkpoints_path + '/' + model_name

learning_rate = 1e-4
n_epochs = 100
batch_size = 8 * tpu_strategy.num_replicas_in_sync
steps_per_execution = 1
steps_per_epoch = 112
validation_steps = 320
gradient_accumulation_steps_per_replica = 8
raster_shape = (256, 256, 1)
input_attributes = np.load('/content/drive/MyDrive/DeepLearning/input_attributes.npy', allow_pickle=True).item()
(eta_mean, eta_std) = input_attributes['eta'].values()
(zeta_mean, zeta_std) = input_attributes['zeta'].values()
(bathy_mean, bathy_std) = input_attributes['bathy'].values()

In [ ]:
def tf_parse(eg):
    """parse an example (or batch of examples, not quite sure...)"""

    # here we re-specify our format
    # you can also infer the format from the data using tf.train.Example.FromString
    # but that did not work
    example = tf.io.parse_example(
        eg[tf.newaxis],
        {
            'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'depth': tf.io.FixedLenFeature([], tf.int64),
            'bathy': tf.io.FixedLenFeature([], tf.string),
            'hs': tf.io.FixedLenFeature([], tf.string),
            'tm01': tf.io.FixedLenFeature([], tf.string),
            'theta0x': tf.io.FixedLenFeature([], tf.string),
            'theta0y': tf.io.FixedLenFeature([], tf.string),
            'eta': tf.io.FixedLenFeature([], tf.float32),
            'zeta': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavex': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavey': tf.io.FixedLenFeature([], tf.float32),
            'mask': tf.io.FixedLenFeature([], tf.string),
        },
    )
    bathy = tf.io.parse_tensor(example["bathy"][0], out_type="float32")
    bathy = tf.ensure_shape(bathy, raster_shape)    # ensure shape, to be able to train the model
    hs = tf.io.parse_tensor(example["hs"][0], out_type="float32")
    hs = tf.ensure_shape(hs, raster_shape)
    tm01 = tf.io.parse_tensor(example["tm01"][0], out_type="float32")
    tm01 = tf.ensure_shape(tm01, raster_shape)
    theta0x = tf.io.parse_tensor(example["theta0x"][0], out_type="float32")
    theta0x = tf.ensure_shape(theta0x, raster_shape)
    theta0y = tf.io.parse_tensor(example["theta0y"][0], out_type="float32")
    theta0y = tf.ensure_shape(theta0y, raster_shape)
    eta = example["eta"]
    zeta = example["zeta"]
    theta_wavex = example["theta_wavex"]
    theta_wavey = example["theta_wavey"]
    # Angles were in degrees instead of radians when dataset created, so adjust theta_wavex and theta_wavey
    theta_wave = (tf.math.atan2(theta_wavey, theta_wavex) + 2*np.pi) % (2*np.pi) * np.pi / 180.
    theta_wavex = tf.math.cos(theta_wave)
    theta_wavey = tf.math.sin(theta_wave)
    # Output mask
    mask = tf.io.parse_tensor(example["mask"][0], out_type="bool")
    mask = tf.cast(mask, dtype="int16")
    mask = tf.ensure_shape(mask, raster_shape)
    # Create mask for input where waterlevel is less than bathymetry
    bathy_mask = tf.math.greater(bathy * bathy_std + bathy_mean, zeta * zeta_std + zeta_mean)
    bathy_mask = tf.cast(bathy_mask, dtype="float32")
    img_input = tf.concat([bathy,bathy_mask],-1)
    attr = tf.stack([eta, zeta, theta_wavex, theta_wavey], axis=1)
    attr = tf.reshape(attr,shape=[-1])
    output = (hs, tm01, theta0x, theta0y)
    output = tf.concat([hs, tm01, theta0x, theta0y], axis=-1)
    #output = hs
    return (img_input, attr), (output, mask)

In [ ]:
def get_files(data_path):
    files = tf.io.gfile.glob(data_path + "/" + "*.tfrecords")
    return files

def get_dataset(files):
    """return a tfrecord dataset with all tfrecord files"""
    dataset =  tf.data.TFRecordDataset(files)
    dataset = dataset.map(tf_parse)
    return dataset

In [ ]:
def AttnBlock2D(x, g, inter_channel):

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1])(x)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1])(g)

    f = Activation('relu')(tf.keras.layers.add([theta_x, phi_g]))

    psi_f = Conv2D(1, [1, 1], strides=[1, 1])(f)

    rate = Activation('sigmoid')(psi_f)

    att_x = tf.keras.layers.multiply([x, rate])

    return att_x


def attention_up_and_concate(down_layer, layer):
    
    in_channel = down_layer.get_shape().as_list()[3]

    up = UpSampling2D(size=(2, 2))(down_layer)
    layer = AttnBlock2D(x=layer, g=up, inter_channel=in_channel // 4)
    
    my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
    
    concate = my_concat([up, layer])
    return concate

In [ ]:
def full_model(cnn_input_shape, mlp_input_shape):
    
    mlp_input = Input(mlp_input_shape)
    cnn_input = Input(cnn_input_shape)
    x = Dense(256, activation='relu')(mlp_input)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    mlp_output = Dense(4, activation='relu')(x)

    x = Conv2D(16, (3,3), padding="same")(cnn_input)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2D(32, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    # Downsampling with stride=2
    x = Conv2D(64, (3,3), padding="same", strides=(2,2))(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    cnn_output = BatchNormalization()(x)

    conv_shape = K.int_shape(cnn_output)

    # Reshape output from MLP to CNN shape to concatenate
    x = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation="relu")(mlp_output)
    x = Reshape((conv_shape[1],conv_shape[2],int(conv_shape[3])))(x)
    
    last_x = Concatenate()([x,cnn_output])

    #################################### Normal output

    x = Conv2D(256, (3,3), padding="same")(last_x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    # Upsampling with stride=2    
    x = Conv2DTranspose(128, (3,3), padding="same", strides=(2,2))(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2DTranspose(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(32, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
  
    normal_output = Conv2DTranspose(4, (3,3), padding="same", activation="linear", name="normal_output")(x)
  
    ############################### Mask output

    x = Conv2D(256, (3,3), padding="same")(last_x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    # Upsampling with stride=2
    x = Conv2DTranspose(128, (3,3), padding="same", strides=(2,2))(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2DTranspose(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(32, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    mask_output = Conv2DTranspose(1, (3,3), padding="same", activation="sigmoid", name="mask_output")(x)

    model = Model(inputs=[cnn_input, mlp_input], outputs = [normal_output, mask_output])
    
    return model



In [ ]:
train_files = get_files(data_path_train)
test_files = get_files(data_path_test)

train_dataset = get_dataset(train_files)
test_dataset = get_dataset(test_files)

#Filter out the bathymetries with heights greater than 100m
def filter_fn(x, y):
    return tf.less_equal(tf.reduce_max(x[0][:,:,0]), (100-bathy_mean)/bathy_std)

train_dataset = train_dataset.filter(filter_fn)
test_dataset = test_dataset.filter(filter_fn)

train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

In [ ]:
#for sample in train_dataset.take(1):
#    print(repr(sample))

In [ ]:
start = time.time()
print(time.ctime(start))
with tpu_strategy.scope():
    model = full_model((256, 256, 2), 4)
    opt = Adam(learning_rate=learning_rate, decay=learning_rate / n_epochs)

    losses = {
	  "normal_output": "mean_squared_error",
  	"mask_output": "binary_crossentropy",
    }
    lossWeights = {"normal_output": 1.0, "mask_output": 1.0}

    model.compile(loss=losses, optimizer=opt, steps_per_execution=steps_per_execution)
    
    callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        #monitor='val_mse',
        mode='max',
        save_best_only=True
    ),
    tf.keras.callbacks.CSVLogger(
        filename=checkpoints_path + '.csv')
    ]
    
    model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs)#, callbacks=callbacks)#, steps_per_epoch=steps_per_epoch)
    #model.save(model_path + '/' + model_name + '.h5')

end = time.time()
print(time.ctime(end))

Wed Nov 24 13:11:04 2021
Epoch 1/100
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


92/92 [==============================] - 574s 6s/step - loss: 54.0266 - normal_output_loss: 53.3551 - mask_output_loss: 0.6715 - val_loss: 53.1851 - val_normal_output_loss: 52.5577 - val_mask_output_loss: 0.6275
Epoch 2/100
92/92 [==============================] - 59s 641ms/step - loss: 50.5701 - normal_output_loss: 49.9521 - mask_output_loss: 0.6180 - val_loss: 40.8876 - val_normal_output_loss: 40.3828 - val_mask_output_loss: 0.5048
Epoch 3/100
92/92 [==============================] - 60s 644ms/step - loss: 47.3462 - normal_output_loss: 46.8170 - mask_output_loss: 0.5292 - val_loss: 37.8303 - val_normal_output_loss: 37.4151 - val_mask_output_loss: 0.4152
Epoch 4/100
92/92 [==============================] - 61s 654ms/step - loss: 45.2666 - normal_output_loss: 44.8281 - mask_output_loss: 0.4385 - val_loss: 35.6726 - val_normal_output_loss: 35.3663 - val_mask_output_loss: 0.3063
Epoch 5/100
92/92 [==============================] - 61s 658ms/step - loss: 42.9808 - normal_output_loss: 42.6

In [ ]:
model = full_model((256, 256, 1), 4)
opt = Adam(learning_rate=learning_rate, decay=learning_rate / n_epochs)

losses = {
	"normal_output": "mean_squared_error",
	"mask_output": "binary_crossentropy",
}
lossWeights = {"normal_output": 1.0, "mask_output": 1.0}

model.compile(loss=losses, optimizer=opt, steps_per_execution=steps_per_execution)

callbacks = [
tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoints_path, 
    save_weights_only=True,
    #monitor='val_mse',
    mode='max',
    save_best_only=True
),
tf.keras.callbacks.CSVLogger(
    filename=checkpoints_path + '.csv')
]

model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs, steps_per_epoch=steps_per_epoch, 
          validation_steps=40)

Epoch 1/100
112/112 [==============================] - 265s 2s/step - loss: 56.8978 - normal_output_loss: 56.2471 - mask_output_loss: 0.6507 - val_loss: 52.3799 - val_normal_output_loss: 51.8493 - val_mask_output_loss: 0.5307
Epoch 2/100
 42/112 [==========>...................] - ETA: 1:41 - loss: 58.2195 - normal_output_loss: 57.6309 - mask_output_loss: 0.5886

KeyboardInterrupt: ignored

In [ ]:
print(start, end)
print("Starting time: ", time.ctime(start))
print("Ending time: ", time.ctime(end))
print("Time elapsed: ", datetime.timedelta(seconds=round(end - start)))
print('')
print("Number of parameters: ", model.count_params())
print("")
model.save(model_path + '/' + model_name + '.h5')

In [ ]:
logging.info("training model...")
# TODO properly compute steps for progress bar (low priority)
steps_per_epoch = len(train_files) * 10 // batch_size

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        monitor='val_mse',
        mode='max',
        save_best_only=True
    )
]
model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs, callbacks=callbacks)

In [ ]:
model.summary()
model.count_params()
#tf.keras.utils.plot_model(model, show_shapes=True)